<a href="https://colab.research.google.com/github/TaegeunSONG/KSIAM_2025/blob/main/INR_sine_activation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fourier Transform

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

t = np.linspace(0, 5, 1000)
f0 = 1
y_true = np.sin(2 * np.pi * f0 * t)

sample_idx = np.arange(0, len(t), 8)
t_sampled = t[sample_idx]
y_sampled = y_true[sample_idx]

plt.plot(t_sampled, y_sampled, 'o-')
plt.show()

In [ ]:
dom = 0.098
dt = t_sampled[1] - t_sampled[0]

oms = []
y_dft = []

for iom in range(200):
    om = dom * iom
    real = 0.0
    imag = 0.0
    for it, time in enumerate(t_sampled):
        real += y_sampled[it] * np.cos(om*time)*dt
        imag += y_sampled[it] * np.sin(om*time)*dt

    y_dft.append(real**2+imag**2)
    oms.append(om)

plt.plot(oms, y_dft)
plt.show()

In [ ]:
# 2. DFT 계산
Y_dft = np.fft.fft(y_sampled, n=512)
freqs = np.fft.fftfreq(len(Y_dft), d=(t_sampled[1] - t_sampled[0]))

Y_dft_mag = np.abs(np.fft.fftshift(Y_dft))
freqs_shifted = np.fft.fftshift(freqs)

# 4. iDFT 및 ML 기반 복원
y_idft = np.fft.ifft(Y_dft).real[:len(t_sampled)]


fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Top: Original Signal
axes[0].plot(t, y_true, label=r"$y(x_n)$ - Ground Truth", alpha=0.6)
axes[0].scatter(t_sampled, y_sampled, label=r"$y(x_n)$ - sampled", color='orange', s=10)
axes[0].set_title("Given signal")
axes[0].set_xlabel("t [s]")
axes[0].set_ylabel("Y(t)")
axes[0].legend()

axes[1].stem(freqs_shifted, Y_dft_mag)
axes[1].set_title("DFT magnitude")
axes[1].set_xlabel(r"$frequency$")
axes[1].set_ylabel(r"$w_i^2$")
# axes[1].set_xlim([0,5])

axes[2].plot(t_sampled, y_sampled, linewidth=3, label=r"$y(x_n)$")
axes[2].plot(t_sampled, y_idft, label=r"$y^{DFT}$")
axes[2].set_title("Reconstructed from iDFT")
axes[2].set_xlabel("t [s]")
axes[2].set_ylabel("Y(t)")
axes[2].legend()

plt.show()


## INR with sine activation

In [ ]:
import os
import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers, initializers
from tensorflow.python.keras.models import load_model

class WarmUpCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, target_learning_rate, warmup_steps, decay_steps):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.target_learning_rate = target_learning_rate
        self.warmup_steps = warmup_steps
        self.decay_steps = decay_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        decay_steps = tf.cast(self.decay_steps, tf.float32)

        return tf.cond(
            step < warmup_steps,
            lambda: self.initial_learning_rate + (self.target_learning_rate - self.initial_learning_rate) * step / warmup_steps,
            lambda: 0.5 * self.target_learning_rate * (1 + tf.cos(np.pi * (step - warmup_steps) / (decay_steps - warmup_steps)))
        )

    def get_config(self):
        return {
            "initial_learning_rate": self.initial_learning_rate,
            "target_learning_rate": self.target_learning_rate,
            "warmup_steps": self.warmup_steps,
            "decay_steps": self.decay_steps,
        }




In [ ]:

initial_lr = 1e-7
target_lr = 0.1
warmup_steps = 100
decay_steps = 200

schedule = WarmUpCosineDecay(
    initial_learning_rate=initial_lr,
    target_learning_rate=target_lr,
    warmup_steps=warmup_steps,
    decay_steps=decay_steps
)

# step 범위
steps = np.arange(0, 1000)
lrs = [schedule(step).numpy() for step in steps]

# 시각화
plt.figure(figsize=(8, 4))
plt.plot(steps, lrs)
plt.title("Learning Rate: WarmUp + Cosine Decay")
plt.xlabel("Training Step")
plt.ylabel("Learning Rate")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

def sine_activation(x):
    return (K.sin(2*np.pi*x))

def model_get(nh = 10):
    model = Sequential()
    model.add(Input(shape=(1,)))
    model.add(Dense(nh, activation=sine_activation))
    model.add(Dense(1, activation='linear'))
    model.summary()
    return model


In [ ]:
model_path = "./model_save/sine.weights.h5"

cb_checkpoint = ModelCheckpoint(
    filepath=model_path,
    monitor='loss',
    save_best_only=True,         # 가장 좋은 모델만 저장
    save_weights_only=True,      # 모델 전체 저장 (구조 + 가중치)
    verbose = 1
)

cb_early_stopping = EarlyStopping(
    monitor='loss', patience=600, restore_best_weights=True
)

lr_schedule = WarmUpCosineDecay(
    initial_learning_rate=initial_lr,
    target_learning_rate=target_lr,
    warmup_steps=warmup_steps,
    decay_steps=decay_steps
)


opt = Adam(learning_rate=lr_schedule)
model = model_get(nh=10)

model.compile(optimizer=opt, loss='mae')

history=model.fit(t_sampled, y_sampled,  epochs=10000, batch_size=10, \
                  callbacks=[cb_checkpoint,cb_early_stopping],verbose=2)



In [ ]:
import matplotlib.pyplot as plt
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")
# plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.yscale('log')
# plt.xlim((0,25))
plt.show()


In [ ]:
model_best = model_get()
model_best.load_weights(model_path)

weight=model_best.get_weights()
layers=model_best.layers

freq = model_best.layers[0].get_weights()[0][0,:]
phase = model_best.layers[0].get_weights()[1]
amp = model_best.layers[1].get_weights()[0][:,0]
bias = model_best.layers[1].get_weights()[1]

In [ ]:
## reconstruction
gen=[]
for it in t_sampled:
    recover = 0
    for i in range(10):
        recover = recover + amp[i] * np.sin(2.0 * np.pi * (freq[i] * it + phase[i]))
    gen.append(recover + bias[0])


fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Top: Original Signal
axes[0].plot(t, y_true, label=r"$y(x_n)$ - Ground Truth", alpha=0.6)
axes[0].scatter(t_sampled, y_sampled, label=r"$y(x_n)$ - sampled", color='orange', s=10)
axes[0].set_title("Given signal")
axes[0].set_xlabel("t [s]")
axes[0].set_ylabel("Y(t)")
axes[0].legend()

axes[1].stem(freq, amp)
axes[1].set_title("Amplitude")
axes[1].set_xlabel(r"$frequency$")
axes[1].set_ylabel(r"$w_i^2$")
# axes[1].set_xlim([-5,5])

axes[2].plot(t, y_true, label=r"$y(x_n)$ - Ground Truth", alpha=0.6)
axes[2].plot(t_sampled, gen, label=r"$y(x_n)$")
axes[2].set_title("Reconstructed via INR")
axes[2].set_xlabel("t [s]")
axes[2].set_ylabel("Y(t)")
axes[2].legend()

plt.show()

## Apply to missing data

In [ ]:
# missing data generation
min_gap_size = 5
max_gap_size = 10
n_gaps = 5

start_indices = np.random.choice(len(y_sampled) - max_gap_size, n_gaps, replace=False)
indices_to_remove = []

for start in start_indices:
    gap_size = np.random.randint(min_gap_size, max_gap_size + 1)
    indices_to_remove.extend(range(start, start + gap_size))

missing_ydata = np.delete(y_sampled, indices_to_remove)
missing_xdata = np.delete(t_sampled, indices_to_remove)



In [ ]:
## DFT

# 2. DFT 계산
Y_dft = np.fft.fft(missing_ydata, n=512)
freqs = np.fft.fftfreq(len(Y_dft), d=(t_sampled[1] - t_sampled[0]))

Y_dft_mag = np.abs(np.fft.fftshift(Y_dft))
freqs_shifted = np.fft.fftshift(freqs)

# 4. iDFT 및 ML 기반 복원
y_idft = np.fft.ifft(Y_dft).real[:len(t_sampled)]

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 6))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

axes[0].plot(t_sampled, y_sampled, label=r"$y(x_n)$")
axes[0].scatter(missing_xdata, missing_ydata, label=r"$y^{DFT}$")
axes[0].set_title("data set")
axes[0].set_xlabel("t [s]")
axes[0].set_ylabel("Y(t)")
axes[0].legend()



axes[1].plot(t_sampled, y_sampled, label=r"$y(x_n)$")
axes[1].plot(t_sampled, y_idft, label=r"$y^{DFT}$")
axes[1].set_title("Reconstructed from iDFT")
axes[1].set_xlabel("t [s]")
axes[1].set_ylabel("Y(t)")
axes[1].legend()

fig.show()

In [ ]:
model_path = "./model_save/sine.weights.h5"

cb_checkpoint = ModelCheckpoint(
    filepath=model_path,
    monitor='loss',
    save_best_only=True,         # 가장 좋은 모델만 저장
    save_weights_only=True,      # 모델 전체 저장 (구조 + 가중치)
    verbose = 1
)

cb_early_stopping = EarlyStopping(
    monitor='loss', patience=600, restore_best_weights=True
)

lr_schedule = WarmUpCosineDecay(
    initial_learning_rate=initial_lr,
    target_learning_rate=target_lr,
    warmup_steps=warmup_steps,
    decay_steps=decay_steps
)


opt = Adam(learning_rate=lr_schedule)
model = model_get(nh=10)

model.compile(optimizer=opt, loss='mae')

history=model.fit(missing_xdata, missing_ydata,  epochs=10000, batch_size=10, \
                  callbacks=[cb_checkpoint,cb_early_stopping],verbose=2)


In [ ]:
model_best = model_get()
model_best.load_weights(model_path)

weight=model_best.get_weights()
layers=model_best.layers

freq = model_best.layers[0].get_weights()[0][0,:]
phase = model_best.layers[0].get_weights()[1]
amp = model_best.layers[1].get_weights()[0][:,0]
bias = model_best.layers[1].get_weights()[1]

## reconstruction
gen=[]
for it in t_sampled:
    recover = 0
    for i in range(10):
        recover = recover + amp[i] * np.sin(2.0 * np.pi * (freq[i] * it + phase[i]))
    gen.append(recover + bias[0])


fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 6))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

axes[0].plot(t_sampled, y_sampled, label=r"$y(x_n)$")
axes[0].scatter(missing_xdata, missing_ydata, label=r"$y^{sampled}$")
axes[0].set_title("data set")
axes[0].set_xlabel("t [s]")
axes[0].set_ylabel("Y(t)")
axes[0].legend()



axes[1].plot(t_sampled, y_sampled, label=r"$y(x_n)$")
axes[1].plot(t_sampled, gen, label=r"$y^{INR}$")
axes[1].set_title("Reconstructed via INR")
axes[1].set_xlabel("t [s]")
axes[1].set_ylabel("Y(t)")
axes[1].legend()

fig.show()

## Apply to super-resolved

In [ ]:
from tensorflow.keras import datasets


(mnist, _), (_, _) = datasets.mnist.load_data()

img = mnist[5]/127.5 - 1 + np.random.normal(0,0.1,mnist[3].shape)
# img = mnist[4]/255 + np.random.normal(0,0.1,mnist[3].shape)
plt.imshow(img, cmap='gray')

In [ ]:
inputs = []
outputs = []
for i in range(28):
    for j in range(28):
      inputs.append([i+1,j+1])
      outputs.append(img[i,j])

inputs = np.array(inputs)
outputs = np.array(outputs)

In [ ]:

def sine_activation(x):
    return (K.sin(2*np.pi*x))

def model_get_super(nh = 10):
    model = Sequential()
    model.add(Input(shape=(2,)))
    model.add(Dense(nh, activation=sine_activation))
    model.add(Dense(1, activation='linear'))
    model.summary()
    return model


In [ ]:
initial_lr = 1e-9
target_lr = 0.1
warmup_steps = 10
decay_steps = 100


model_path = "./model_save/sine.weights.h5"

cb_checkpoint = ModelCheckpoint(
    filepath=model_path,
    monitor='loss',
    save_best_only=True,         # 가장 좋은 모델만 저장
    save_weights_only=True,      # 모델 전체 저장 (구조 + 가중치)
    verbose = 1
)

cb_early_stopping = EarlyStopping(
    monitor='loss', patience=600, restore_best_weights=True
)

lr_schedule = WarmUpCosineDecay(
    initial_learning_rate=initial_lr,
    target_learning_rate=target_lr,
    warmup_steps=warmup_steps,
    decay_steps=decay_steps
)

reduce_lr = ReduceLROnPlateau(
    monitor='loss',
    factor=0.5,
    patience=30,
    min_lr=1e-9,
    verbose=1
)


nh=100

# opt = Adam(learning_rate=lr_schedule)
opt = Adam()
model = model_get_super(nh)

model.compile(optimizer=opt, loss='mse')

history=model.fit(inputs, outputs, epochs=10000, batch_size=14, \
                  callbacks=[cb_checkpoint,cb_early_stopping,reduce_lr],verbose=2)


In [ ]:
model_best = model_get_super(nh)
model_best.load_weights(model_path)

weight=model_best.get_weights()
layers=model_best.layers

freq_x = model_best.layers[0].get_weights()[0][0,:]
freq_y = model_best.layers[0].get_weights()[0][1,:]
phase = model_best.layers[0].get_weights()[1]
amp = model_best.layers[1].get_weights()[0][:,0]
bias = model_best.layers[1].get_weights()[1]

## reconstruction
gen=[]
x_sample = np.arange(1,28,0.1)
y_sample = np.arange(1,28,0.1)
resolved_img = np.zeros((x_sample.shape[0],y_sample.shape[0]))

for ix, xcord in enumerate(x_sample):
    for iy, ycord in enumerate(y_sample):
        recover = 0
        for i in range(nh):
            recover = recover + amp[i] * np.sin(2.0 * np.pi * (freq_x[i] * xcord + freq_y[i] * ycord + phase[i]))
        resolved_img[ix,iy] = recover + bias[0]


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(6, 8))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

axes[0].imshow(img, cmap='gray')
axes[0].set_title("oiginal")
axes[0].axis('off')



axes[1].imshow(resolved_img, cmap='gray')
axes[1].set_title("Reconstructed via INR")
axes[1].axis('off')

fig.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(6, 8))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

axes[0].imshow(img, cmap='gray')
axes[0].set_title("oiginal")
axes[0].set_xlim([5,14])
axes[0].set_ylim([21,12])

# axes[0].axis('off')



axes[1].imshow(resolved_img, cmap='gray')
axes[1].set_title("Reconstructed via INR")
axes[1].set_xlim([50,140])
axes[1].set_ylim([210,120])
# axes[1].axis('off')

fig.show()